In [1]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from custom_assets import colors


#for jupyter
import os

pio.renderers.default = (
    "browser"  # modify this to plot on something else besides browser
)


In [2]:
#script_dir = Path(__file__).parent  
#data_path = (script_dir / "../../data").resolve()

# for jupyter
script_dir = os.path.abspath('')

import os 
# for silly jupyter
data_path = os.path.join(script_dir, "../../data/")

# Modify the variables below to plot your desired play
input_data_loc = "train/input_2023_w01.csv"
output_data_loc = "train/output_2023_w01.csv"

game_id = 2023090700
play_id = 101
supplementary_file = "supplementary_data.csv"

#tracking_file = data_path / input_data_file
# for silly jupyter
input_data_file = os.path.join(data_path, input_data_loc)
output_data_file = os.path.join(data_path, output_data_loc)
info_file = os.path.join(data_path, supplementary_file)

print(input_data_file)
print(output_data_file)
print(info_file)

/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/train/input_2023_w01.csv
/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/train/output_2023_w01.csv
/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/supplementary_data.csv


In [3]:
df_input = pd.read_csv(input_data_file)
df_output = pd.read_csv(output_data_file)
df_plays = pd.read_csv(info_file)

/var/folders/xj/g4f3qxj923dg9hvptdpcl3mr0000gn/T/ipykernel_63351/3267752177.py:3: DtypeWarning:

Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
df_input_merged = df_input.merge(df_plays, on=["game_id", "play_id"])
df_output_merged = df_output.merge(df_plays, on=["game_id", "play_id"])

print(df_input_merged.columns)

drop_list = ['penalty_yards',
       'pre_penalty_yards_gained', 'yards_gained', 'expected_points',
       'expected_points_added', 'pre_snap_home_team_win_probability',
       'pre_snap_visitor_team_win_probability',
       'home_team_win_probability_added', 'visitor_team_win_probility_added', 'play_nullified_by_penalty']

df_input_final = df_input_merged.drop(drop_list, axis=1)
df_output_merged = df_output_merged.drop(drop_list, axis=1)

Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'season',
       'week', 'game_date', 'game_time_eastern', 'home_team_abbr',
       'visitor_team_abbr', 'play_description', 'quarter', 'game_clock',
       'down', 'yards_to_go', 'possession_team', 'defensive_team',
       'yardline_side', 'yardline_number', 'pre_snap_home_score',
       'pre_snap_visitor_score', 'play_nullified_by_penalty', 'pass_result',
       'pass_length', 'offense_formation', 'receiver_alignment',
       'route_of_targeted_receiver', 'play_action', 'dropback_type',
       'dropback_distance', 'pass_location_type', 'defenders_in_the_box',
       'team_coverage_man_zone', 'team_coverage_type', 'penalty_yards',
       'pre_

In [20]:
df_output_merged[(df_output_merged["frame_id"] == 1)].iloc[0]

game_id                                                              2023090700
play_id                                                                     101
nfl_id                                                                    46137
frame_id                                                                      1
x                                                                         56.22
y                                                                         17.28
season                                                                     2023
week                                                                          1
game_date                                                            09/07/2023
game_time_eastern                                                      20:20:00
home_team_abbr                                                               KC
visitor_team_abbr                                                           DET
play_description              (14:25) (S

In [5]:
# Create a mapping of player roles from pre-pass data
# Each player has one role per play
role_mapping = df_input_final[['game_id', 'play_id', 'nfl_id', 'player_role']].drop_duplicates()
df_output_final = df_output_merged.merge(
    role_mapping,
    on=['game_id', 'play_id', 'nfl_id'],
    how='left'
)


In [11]:
df_input_focused = df_input_final[
    (df_input_final["play_id"] == play_id) & (df_input_final["game_id"] == game_id)
]

df_output_focused = df_output_final[
    (df_output_final["play_id"] == play_id) & (df_output_final["game_id"] == game_id)
]


# Concat the two dataframes - merge the two segments of the play into one
df_output_focused['frame_id'] = df_output_focused['frame_id'] + df_input_focused['frame_id'].max()
df_focused = pd.concat([df_input_focused, df_output_focused], ignore_index=True)


/var/folders/xj/g4f3qxj923dg9hvptdpcl3mr0000gn/T/ipykernel_63351/3774775338.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'season',
       'week', 'game_date', 'game_time_eastern', 'home_team_abbr',
       'visitor_team_abbr', 'play_description', 'quarter', 'game_clock',
       'down', 'yards_to_go', 'possession_team', 'defensive_team',
       'yardline_side', 'yardline_number', 'pre_snap_home_score',
       'pre_snap_visitor_score', 'pass_result', 'pass_length',
       'offense_formation', 'receiver_alignment', 'route_of_targeted_receiver',
       'play_action', 'dropback_type', 'dropback_distance',
       'pass_location_type', 'defenders_in_the_box', 'team_coverage_man_zone',
       'team_coverage_type'],
      dtype='object')

In [18]:
df_focused[(df_focused["player_to_predict"] == True) & (df_focused["frame_id"] == 1)].iloc[0]

game_id                                                              2023090700
play_id                                                                     101
player_to_predict                                                          True
nfl_id                                                                    46137
frame_id                                                                      1
play_direction                                                            right
absolute_yardline_number                                                   42.0
player_name                                                         Justin Reid
player_height                                                               6-1
player_weight                                                             204.0
player_birth_date                                                    1997-02-15
player_position                                                              SS
player_side                             